In [12]:
import langchain
print(langchain.__version__)

1.2.0


In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
import dotenv
from dotenv import load_dotenv
import os
from langchain_core.tools import tool,Tool
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
# Import FakeListLLM for testing
from langchain_community.llms import FakeListLLM


In [14]:
load_dotenv()
serp_api_key=os.getenv("SERP_API_KEY")
google_api_key=os.getenv("GOOGLE_API_KEY")

In [15]:
prompt='''You are an expert educator and quiz master for advanced technical students.
Your goal is to create a quiz based strictly on the following text context.
Text Context:   "{text}"

Instructions to be followed:
1. Create {num_questions} multiple-choice questions(MCQs) based only on the Text above"
2. For each question, provide 4 options (A, B, C, D)
3. Identfiy the correct answer
4. Ensure the questions test understanding, not just simple words matching.
5. Difficulty level : {difficulty} (eg. easy,medium or hard)

Output Format: 
You must output a strictly valid JSON string. Do not output markdown code blocks.
The JSON format must be list of objects like this: 
[
    {{
        "question": "The question text here?",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_answer": "Option B",
        "explanation": "A brief explanation of why B is correct."
    }}
]
'''

In [16]:
prompt_template=PromptTemplate(input_variables=["text","num_questions","difficulty"],template=prompt)

In [17]:
final_prompt=(prompt_template.format(text='''International Debut: He made his ODI debut for India on August 18, 2008, against Sri Lanka.
World Cup Success: Kohli was a key member of the Indian team that won the 2011 ODI World Cup, the 2013 ICC Champions Trophy, and the 2024 T20 World Cup.
Most ODI Centuries: He holds the world record for the most centuries in One Day Internationals (ODIs), surpassing Sachin Tendulkar's long-standing record with his 50th ODI century during the 2023 World Cup.
Record Run Scorer: He is the fastest batsman to reach multiple thousand-run milestones in ODIs, including 8,000, 10,000, 12,000, and 14,000 runs.
Successful Test Captain: As India's most successful Test captain, he led the team to a historic Test series win in Australia in 2018-19 and helped them maintain the #1 Test ranking for a long period.
ICC Awards: His numerous accolades include the Sir Garfield Sobers Trophy (ICC Cricketer of the Year) in 2017 and 2018, and the ICC Men's ODI Cricketer of the Decade (2011–2020).
IPL Career: In the Indian Premier League (IPL), he has played exclusively for the Royal Challengers Bengaluru (RCB) since 2008 and led them to their first title in the 2025 season.
Retirement Announcements: Kohli retired from T20 International cricket in 2024 after India's T20 World Cup victory and announced his retirement from Test cricket in May 2025.
Personal Life: He is married to popular Bollywood actress Anushka Sharma, and they have two children together, a daughter named Vamika and a son named Akaay.
Off-Field Influence: Known for his discipline and passion, he is a role model for aspiring athletes and a highly influential global figure with a significant net worth from endorsements and business ventures.'''
, num_questions=10,difficulty='easy'))

In [18]:
# MOCK (DUMMY) IMPLEMENTATION
# This replaces the real Gemini call with a fake one to save API credits during testing.

USE_MOCK = True

dummy_response_json = """
[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]
"""

if USE_MOCK:
    print("Using FAKE LLM for testing...")
    # FakeListLLM expects a list of responses to cycle through
    client = FakeListLLM(responses=[dummy_response_json])
    response = client.invoke(final_prompt) 
    # Note: FakeListLLM returns string directly in some versions, or object in others. 
    # Invoke typically returns a string for legacy LLMs or BaseMessage for ChatModels.
    # Let's handle both just in case.
    if hasattr(response, 'content'):
        print(response.content)
    else:
        print(response)
else:
    print("Using REAL Gemini LLM...")
    client = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    response = client.invoke(final_prompt)
    print(response.content)

Using FAKE LLM for testing...

[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]



In [23]:
messages=client.invoke(prompt.format(text='Hello sir how are you',num_questions=2,difficulty='easy'))

In [24]:
print(messages)


[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]



In [11]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader(r"C:\ALL PROJECTS\BrainBolt\data\pdfs\AI_Powered_Document_Search_RAG_Overview.pdf")
pages=loader.load()
full_text="\n".join([page.page_content for page in pages])
print(full_text)



AI-Powered Document Search & Question Answering (RAG System)
Purpose & Vision
This project aims to develop an AI-powered system that enables users to upload, search, and
query documents with intelligent, context-aware responses. The system integrates Retrieval-
Augmented Generation (RAG) with Google Gemini, combining local vector search with a generative
language model to produce accurate and grounded answers.
Core Concept: Retrieval-Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) bridges the gap between retrieval-based search and
generative AI. It retrieves the most relevant document chunks through vector similarity search
and feeds them to a large language model (LLM), ensuring responses are contextually accurate,
citeable, and based on real data rather than hallucinations. This creates a balance between
factual precision and generative reasoning.
System Architecture
1. Document ingestion: Users can upload PDF, DOCX, TXT, or provide URLs.
2. Text extraction: Raw text 

In [10]:
import os
print(os.path.isfile("C:\ALL PROJECTS\BrainBolt\data\pdfs\AI_Powered_Document_Search_RAG_Overview.pdf"))


True
